### Import models and load data

In [3]:
from torchvision import datasets
import torch
import dlc_practical_prologue as prologue

# Import all models
from BaseNet import *
from ConvNet1 import *
#from NetSharing import *

mini_batch_size = 1000
nb_epochs = 300
nb_runs = 2
eta = 0.001

train_input, train_target, train_classes, test_input, test_target, test_classes = \
    prologue.generate_pair_sets(nb=1000)

/usr/local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [4]:
train_classes.shape

torch.Size([1000, 2])

In [8]:
a = torch.tensor([[1,2],[3,4],[5,6]])
a

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [11]:
a.narrow(1,0,1)

tensor([[1],
        [3],
        [5]])

In [13]:
a.narrow(1,1,1)

tensor([[2],
        [4],
        [6]])

### 1channel2images

In [2]:
from importlib import reload
import _1channel2images
reload(_1channel2images)
from _1channel2images import *

print("Working with 1channel2images framework, nb_classes = ", nb_classes)

#model = BaseNet1C(nb_classes)
model_1C = ConvNet1_1C(nb_classes)
optimizer_1C = torch.optim.SGD(model_1C.parameters(), lr=eta, momentum=0.95)
test_results_1C = multiple_training_runs(model_1C, 2, optimizer_1C, train_input, train_classes,
                                      test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)

Working with 1channel2images framework, nb_classes =  10
phase: train, epoch: 100, loss: 1.44493, acc: 0.7915
phase: val, epoch: 100, loss: 1.42741, acc: 0.7945
phase: train, epoch: 200, loss: 0.27240, acc: 0.9605
phase: val, epoch: 200, loss: 0.26991, acc: 0.9610
phase: train, epoch: 300, loss: 0.09784, acc: 0.9925
phase: val, epoch: 300, loss: 0.09711, acc: 0.9925
Training complete in 7 min 40 s
Best val acc: 0.9925
phase: train, epoch: 100, loss: 1.36705, acc: 0.8015
phase: val, epoch: 100, loss: 1.35128, acc: 0.8050
phase: train, epoch: 200, loss: 0.24406, acc: 0.9655
phase: val, epoch: 200, loss: 0.24171, acc: 0.9655
phase: train, epoch: 300, loss: 0.08303, acc: 0.9935
phase: val, epoch: 300, loss: 0.08235, acc: 0.9945
Training complete in 6 min 54 s
Best val acc: 0.9945
Overwriting file


In [3]:
# This is just for visualization of our results but it will have to be taken away for the report 
# since we can't use any additional libraries
import pandas as pd

print(test_results_1C)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)
data = pd.read_csv('1channel2images.csv')
data.head()

(436.9516521692276, 32.183775053303684, 0.8707499999999999, 0.0017677669529664096, 0.912, 0.004242640687119289)
Overwriting file


,Model,Number of parameters,Training time,Mean digits accuracy (test set),Std digits accuracy,Mean accuracy (test set),Std accuracy
0,ConvNet1_1C,72226,436.95,0.8707,0.0018,0.912,0.0042


### 2channels1image

In [5]:
import _2channels1image
reload(_2channels1image)
from _2channels1image import *

print("Working with 2channels1image framework, nb_classes = ", nb_classes)

model_2C = ConvNet1_2C(nb_classes)
optimizer_2C = torch.optim.SGD(model_2C.parameters(), lr=0.00001, momentum=0.95)  
test_results_2C = multiple_training_runs(model_2C, 2, optimizer_2C, train_input, train_target,
                           test_input, test_target, mini_batch_size, nb_epochs)
write_to_csv('2channels1image.csv', model_2C, test_results_2C)

Working with 2channels1image framework, nb_classes =  1
phase: train, epoch: 100, loss: 0.64736, acc: 0.6630
phase: val, epoch: 100, loss: 0.64125, acc: 0.6690
phase: train, epoch: 200, loss: 0.54879, acc: 0.7190
phase: val, epoch: 200, loss: 0.54821, acc: 0.7200
phase: train, epoch: 300, loss: 0.49894, acc: 0.7560
phase: val, epoch: 300, loss: 0.49851, acc: 0.7560
Training complete in 1 min 26 s
Best val acc: 0.7560
phase: train, epoch: 100, loss: 0.63745, acc: 0.6730
phase: val, epoch: 100, loss: 0.63124, acc: 0.6760
phase: train, epoch: 200, loss: 0.53927, acc: 0.7260
phase: val, epoch: 200, loss: 0.53869, acc: 0.7280
phase: train, epoch: 300, loss: 0.48994, acc: 0.7560
phase: val, epoch: 300, loss: 0.48954, acc: 0.7560
Training complete in 1 min 37 s
Best val acc: 0.7560


In [6]:
print(test_results_2C)
write_to_csv('2channels1image.csv', model_2C, test_results_2C)
data = pd.read_csv('2channels1image.csv')
data.head()

(91.07980024814606, 7.846400394134911, 0.7045, 0.002121320343559566)


,Model,Number of parameters,Training time,Mean accuracy (test set),Std accuracy
0,ConvNet1_2C,70705,91.08,0.7045,0.0021


### Weight Sharing

In [10]:
import weight_sharing
reload(weight_sharing)
from weight_sharing import *

print("Working with weight_sharing framework")

model_ws = NetSharing1()
optimizer_ws = torch.optim.SGD(model_ws.parameters(), lr=0.00001, momentum=0.95)  
test_results_ws = multiple_training_runs(model_ws, nb_runs, optimizer_ws, train_input, train_target,
                           test_input, test_target, mini_batch_size, nb_epochs)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)

Working with weight_sharing framework
phase: train, epoch: 100, loss: 0.50720, acc: 0.8010
phase: val, epoch: 100, loss: 0.49663, acc: 0.7980
phase: train, epoch: 200, loss: 0.38888, acc: 0.8340
phase: val, epoch: 200, loss: 0.38819, acc: 0.8340
phase: train, epoch: 300, loss: 0.33217, acc: 0.8570
phase: val, epoch: 300, loss: 0.33173, acc: 0.8570
Training complete in 2 min 34 s
Best val acc: 0.8570
phase: train, epoch: 100, loss: 0.49602, acc: 0.8040
phase: val, epoch: 100, loss: 0.48593, acc: 0.8000
phase: train, epoch: 200, loss: 0.37968, acc: 0.8370
phase: val, epoch: 200, loss: 0.37899, acc: 0.8370
phase: train, epoch: 300, loss: 0.32445, acc: 0.8610
phase: val, epoch: 300, loss: 0.32403, acc: 0.8610
Training complete in 2 min 40 s
Best val acc: 0.8610


In [11]:
print(test_results_ws)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)
data = pd.read_csv('weightsharing.csv')
data.head()

(157.0215289592743, 4.446958156281669, 0.768, 0.0014142135623730963)


,Model,Number of parameters,Training time,Mean accuracy (test set),Std accuracy
0,NetSharing1,70318,157.02,0.768,0.0014


### Auxiliary Losses

In [60]:
import auxiliary_losses
reload(auxiliary_losses)
from auxiliary_losses import *
import Incept1
reload(Incept1)
from Incept1 import *
nb_epochs = 300
print("Working with auxiliary losses framework")

model_aux = Incept1()
optimizer_aux = torch.optim.SGD(model_aux.parameters(), lr=0.00001, momentum=0.95)  
test_results_aux = multiple_training_runs(model_aux, nb_runs, optimizer_aux, train_input, train_target, train_classes, 
                                          test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('auxiliary_losses.csv', model_aux, test_results_aux)

Working with auxiliary losses framework
train_input.shape =  torch.Size([1000, 2, 14, 14])
train_classes.shape =  torch.Size([1000, 2])
train_classes[0] =  tensor([9, 3])
train_target.shape =  torch.Size([1000, 1])
phase: train, epoch: 100, loss: 2.20260, acc: 0.6860
phase: val, epoch: 100, loss: 2.19653, acc: 0.6870
phase: train, epoch: 200, loss: 1.93689, acc: 0.7290
phase: val, epoch: 200, loss: 1.93538, acc: 0.7290
phase: train, epoch: 300, loss: 1.82158, acc: 0.7450
phase: val, epoch: 300, loss: 1.82069, acc: 0.7460
Training complete in 2 min 35 s
Best val acc: 0.7460
output_to_pred =  tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1], dtype=torch.uint8)
target =  tensor([1, 0, 0, 1, 0, 1, 0, 1, 1, 1])
nb_errors =  307
acc_pairs =  0.6930000000000001
train_input.shape =  torch.Size([1000, 2, 14, 14])
train_classes.shape =  torch.Size([1000, 2])
train_classes[0] =  tensor([9, 3])
train_target.shape =  torch.Size([1000, 1])
phase: train, epoch: 100, loss: 2.17891, acc: 0.6870
phase: val, epoch:

In [62]:
print(test_results_aux)
write_to_csv('auxiliary_losses.csv', model_aux, test_results_aux)
data = pd.read_csv('auxiliary_losses.csv')
data.head()

(160.96547901630402, 8.627074330809831, 0.6915, 0.002121320343559723)
Overwriting file


,Model,Number of parameters,Training time,Mean accuracy (test set),Std accuracy
0,Incept1,175027,160.97,0.6915,0.0021


### Weight Sharing Model (Youssef)

In [2]:
print(train_input.shape, train_target.shape, train_classes.shape, test_input.shape, test_target.shape, test_classes.shape)

torch.Size([1000, 2, 14, 14]) torch.Size([1000]) torch.Size([1000, 2]) torch.Size([1000, 2, 14, 14]) torch.Size([1000]) torch.Size([1000, 2])


In [6]:
from torch.autograd import Variable
from WSharingModel import *
from torch import nn, optim

In [7]:
# normalize it
mean, std = train_input.mean(), train_input.std() 
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)
train_input_Net3 = train_input.view(-1, 1, 14, 14)
train_target_Net3 = train_classes.flatten()
train_input, train_target, train_classes = Variable(train_input), Variable(train_target), Variable(train_classes)
test_input, test_target = Variable(test_input), Variable(test_target)

In [8]:
    NB_EPOCHS = 25
    MINI_BATCH_SIZE = 100
    model = NetSharing1()
    #models = [NetAuxiliary1]
    optimizers = [optim.SGD]
    learning_rates = [1e-1, 1e-2, 1e-3]
    for optimizer in optimizers:
            for learning_rate in learning_rates:
               
                training_time = train_model(model, optimizer(model.parameters(), lr=learning_rate), NB_EPOCHS, \
                        train_input, train_target, MINI_BATCH_SIZE)
 
                print('model: {:>13}, criterion: {:>10}, optimizer: {:>10}, learning rate: {:6}, num epochs: {:3}, '
                    'mini batch size: {:3}, training time: {:5.2f}, train error: {:5.2f}%, test error: {:5.2f}%'.format(
                    model.__class__.__name__,
                    model.criterion.__class__.__name__,
                    optimizer.__name__,
                    learning_rate,
                    NB_EPOCHS,
                    MINI_BATCH_SIZE,
                    training_time,
                    compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100,
                    compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100
                    )
                )

0.6668369174003601
0.6389387249946594
0.5997783541679382
0.5608848929405212
0.47221630811691284
0.43288353085517883
0.41329720616340637
0.3842991888523102
0.35745057463645935
0.32919201254844666
0.2964293956756592
0.3015657365322113
0.257221519947052
0.24459552764892578
0.24022550880908966
0.22103363275527954
0.20277614891529083
0.18759697675704956
0.2066190391778946
0.16215385496616364
0.1551346778869629
0.13588294386863708
0.13199284672737122
0.11133017390966415
0.15101686120033264
model:   NetSharing1, criterion: CrossEntropyLoss, optimizer:        SGD, learning rate:    0.1, num epochs:  25, mini batch size: 100, training time:  8.00, train error:  4.00%, test error: 16.20%
0.06339611858129501
0.06098883971571922
0.06073635071516037
0.060218919068574905
0.059233374893665314
0.05807508900761604
0.057028498500585556
0.05593902990221977
0.05480261147022247
0.053654905408620834
0.052507515996694565
0.05141366645693779
0.05030186101794243
0.04922853410243988
0.048247165977954865
0.04726